# Importation de la fonction pipeline

In [29]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_path)

from Application.data_processing import pipeline

# Définition des paramètres

Il faut défnir le chemin où sont stockés les données. <br>
Quel est la variable cible. <br>
A partir de quelle variable on veut créer nos subsets. <br>
Comment on nomme l'expérience dans mlflow. <br>

In [30]:
chemin = '../Data/datanba/'
cible ='Rounds'
critère = 'Pace'
experience_name ='Classification_Playoffs'

# Lancement du pipeline

On peut uniquement lancer la fonction pipeline qui s'occupe de tout.

In [ ]:
resultat = pipeline(chemin,cible,critère,experience_name)
print(resultat)

# Tout le processus

Ou bien c'est possible, si l'on veut passer au travers de certaines étapes, de reprendre chacunes des fonctions et executer celle que l'on souhaite.

# Importation des packages

In [31]:
from Infrastucture.data_loading import load_data
from Infrastucture.data_cleaning import clean_data, subsets
from Domain.data_preprocessing import preprocessing
from Domain.data_modelisation import ModelTrainer, model_testing
from Application.data_processing import sauvegarder_modele, charger_modele, pipeline
from sklearn.model_selection import train_test_split

# Définition des paramètres

In [33]:
chemin = '../Data/datanba/'
cible ='Rounds'
critère = 'Pace'
experience_name ='Classification_Playoffs'

# Chargement des données

In [34]:
df = load_data(chemin)

# Traitement des données

In [35]:
df = clean_data(df)

Attend./G: 11 valeurs nulles
AS: 29 valeurs nulles
['Boston Celtics' 'Los Angeles Lakers' 'Philadelphia 76ers'
 'Seattle SuperSonics' 'Phoenix Suns*' 'Atlanta Hawks*' 'Milwaukee Bucks*'
 'Kansas City Kings*' 'Houston Rockets*' 'San Antonio Spurs*'
 'Washington Bullets*' 'New York Knicks' 'Portland Trail Blazers*'
 'Cleveland Cavaliers' 'Indiana Pacers' 'San Diego Clippers'
 'New Jersey Nets' 'Chicago Bulls' 'Denver Nuggets'
 'Golden State Warriors' 'Utah Jazz' 'Detroit Pistons' 'Boston Celtics*'
 'Philadelphia 76ers*' 'Los Angeles Lakers*' 'New York Knicks*'
 'Chicago Bulls*' 'Indiana Pacers*' 'Washington Bullets' 'Atlanta Hawks'
 'Dallas Mavericks' 'Seattle SuperSonics*' 'Denver Nuggets*'
 'New Jersey Nets*' 'Portland Trail Blazers' 'Kansas City Kings'
 'Houston Rockets' 'Detroit Pistons*' 'Utah Jazz*' 'Dallas Mavericks*'
 'San Antonio Spurs' 'Cleveland Cavaliers*' 'Los Angeles Clippers'
 'Sacramento Kings*' 'Phoenix Suns' 'Golden State Warriors*'
 'Sacramento Kings' 'Charlotte Hornet

# Séparation en subsets

In [36]:
subset_sup, subset_inf = subsets(df, critère)

Si on veut travailler sur un seul subset par exemple, on le défini.

In [37]:
subset = subset_sup

# Préprocessing

In [38]:
X, y, scaler = preprocessing(subset, cible)

# Modélisation

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
trainer = ModelTrainer()

# Test des performances des modèles

In [ ]:
best_model, scores = trainer.train_and_evaluate(X_train, y_train, X_test, y_test)

In [ ]:
model_testing(best_model,X_test,y_test)

# Sauvegarder et charger dans mlflow

In [ ]:
sauvegarder_modele(best_model, f"modele_sklearn_{subset}", experience_name, subset)

In [ ]:
charger_modele(f"modele_sklearn_{subset}")